In [2]:
# imports
%pylab inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
import random
from numpy import linalg as LA
from scipy.fftpack import dct
import matplotlib.pyplot as plt
from skimage.transform import rescale, resize, downscale_local_mean


Populating the interactive namespace from numpy and matplotlib


/home/kapi/anaconda3/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['random', 'resize']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [9]:
def dist(img1, img2):
    res = numpy.linalg.norm(img1 - img2, ord = 1)
    return res

def dist_complex(img1, img2):
    res = 0
    for i in range(len(img1)):
        res += abs(img1[i] - img2[i])
    return res;

def scale(img, mult):
    res = rescale(img, 1 / mult, anti_aliasing=False).flatten() * 200
        
    return res.astype(int)

#initialization
def load_pic():
    class_count = 40
    image_count = 10
    imgs = []
    for i in range(class_count):
        imgs.append([])
        for j in range(image_count):
            address = "data/s" + str(i+1) + "/" + str(j+1) + ".pgm"
            imgs[i].append(mpimg.imread(address))
    return imgs

#set test, train

def test_train(imgs, count):
    train = []
    test = []
    for i in imgs:
        train.append(i[:count])
        test.append(i[count:])
    return test, train

def dct2(img2, size):
    img = np.array(img2)
    m = dct(dct(img.T, norm='ortho').T, norm='ortho')
    res = []
    for i in range(size[0]):
        for j in range(size[1]):
            res.append(m[i][j])
    return np.array(res) 

def hist(img, count):
    res = []
    for i in range(count):
        res.append(0)
    for i in img:
        for j in i:
            res[int((j/255)*count)] += 1
    return res;

def vote(result, class_count):
    voted = []
    for i in range (len(result[0])):
        value = [0]*class_count
        for j in result:
            value[j[i]] += 1
        max_id = 0
        max_value = value[0]
        for i in range(len(value)):
            if (value[i] > value[max_id]):
                max_id = i
                max_value = value[i]
        voted.append(max_id)
    return voted

def fr(test, train):
    res = 0
    res_list = []
    for i in range(len(test)):
        for j in range(len(test[0])):
            i_id = 0
            image_m = test[i][j]
            i_dist_min = dist(image_m, train[0][0])
            for q in range(len(train)):
                for p in range(len(train[0])):
                    i_dist = dist(image_m, train[q][p])
                    if (i_dist < i_dist_min):
                        i_dist_min = i_dist
                        i_id = q
            if (i_id == i):
                res += 1
            res_list.append(i_id)
    print(res/(10-count_to_load)/40)
    return res_list
            
def fr_pic(test, train):
    res = 0
    res_list = []
    for i in range(len(test)):
#         res_list.append([])
        for j in range(len(test[0])):
            i_id = 0
            i_num = 0
            image_m = test[i][j]
            i_dist_min = dist(image_m, train[0][0])
            for q in range(len(train)):
                for p in range(len(train[0])):
                    i_dist = dist(image_m, train[q][p])
                    if (i_dist < i_dist_min):
                        i_dist_min = i_dist
                        i_num = p
                        i_id = q
            if (i_id == i):
                res += 1
            res_list.append([i_id, i_num])
#     print(res/(10-count_to_load)/40)
    return res_list

In [19]:
def get_imgs_scaled(imgs, mult):
    res = []
    for i in range(class_count):
        res.append([])
        for j in range(image_count):
            res[i].append(scale(imgs[i][j], mult))
    return res

In [66]:
def get_pca(imgs):
    V = np.cov(imgs.T)
    values, vectors = eig(V)
    P = vectors.T.dot(m_imgs.T).T
    return P

def get_pca_imgs(imgs, count):
    imgs_prePCA = get_imgs_scaled(imgs, 3)
    reshapeY = 37
    reshapeX = 31
    m_imgs = np.reshape(imgs_prePCA, (total_imgs , len(imgs_prePCA[0][0])))
    mid = (m_imgs.sum(axis=0) / total_imgs).astype(int)
    m_imgs = m_imgs - mid
    pca = get_pca(m_imgs)
    main_values = 30
    res = []
    count = 0
    for i in range(class_count):
        res.append([])
        for j in range(image_count):
            res[i].append(pca[count][:30])
            count += 1
    return res
        

In [88]:
imgs = load_pic()
count_to_load = 5
image_count = 10
class_count = 40
total_imgs = image_count*class_count
result = []
for i in range(10):
    result.append([])
    for j in range(5):
        result[i].append([])
        
        
imgs_scaled = []
mult = 4
for i in range(class_count):
    imgs_scaled.append([])
    for j in range(image_count):
        imgs_scaled[i].append(scale(imgs[i][j], mult))
    
imgs_cos = []
size = [5, 5]
for i in range(class_count):
    imgs_cos.append([])
    for j in range(image_count):
        imgs_cos[i].append(dct2(imgs[i][j], size))

In [11]:
mult = 4
test_count = 10
for i in range(1,test_count):
    count_to_load = i
    test, train = test_train(imgs_scaled, count_to_load)
    result[i-1][0] = (fr(test, train))

0.7472222222222222
0.853125
0.8821428571428571
0.9166666666666666
0.945
0.98125
0.975
0.975
0.95


In [12]:
for i in range(1,test_count):
    count_to_load = i
    test, train = test_train(imgs_cos, count_to_load)
    result[i-1][1] = (fr(test, train))

0.725
0.81875
0.85
0.8875
0.9
0.96875
0.9666666666666666
0.975
0.975


In [70]:
imgs_pca = get_pca_imgs(imgs, 30)

In [71]:
for i in range(1,test_count):
    count_to_load = i
    test, train = test_train(imgs_pca, count_to_load)
    result[i-1][0] = (fr(test, train))

0.7083333333333333
0.825
0.85
0.8875
0.8949999999999999
0.95
0.95
0.9375
0.925


In [110]:
def get_pca(imgs):
    V = np.cov(imgs.T)
    values, vectors = eig(V)
    P = vectors.T.dot(m_imgs.T).T
    return P

def get_pca_imgs(imgs, count):
    imgs_prePCA = get_imgs_scaled(imgs, 3)
    reshapeY = 37
    reshapeX = 31
    m_imgs = np.reshape(imgs_prePCA, (total_imgs , len(imgs_prePCA[0][0])))
    mid = (m_imgs.sum(axis=0) / total_imgs).astype(int)
    m_imgs = m_imgs - mid
    pca = get_pca(m_imgs)
    main_values = 30
    res = []
    count = 0
    for i in range(class_count):
        res.append([])
        for j in range(image_count):
            res[i].append(pca[count][:30])
            count += 1
    return res
        
mult = 1
reshapeY = len(rescale(imgs[0][0],mult))
reshapeX = len(rescale(imgs[0][0],mult)[0])
print(reshapeY, reshapeX)
# reshapeX = 31
imgs_prePCA = get_imgs_scaled(imgs, mult)
m_imgs = np.reshape(imgs_prePCA, (total_imgs , len(imgs_prePCA[0][0])))
m_imgs = np.array([np.reshape(i, (reshapeY, reshapeX)) for i in m_imgs])
print(m_imgs.shape)
# pca = get_pca(m_imgs)

m_cov = np.sum([np.dot(i, i.T) for i in m_imgs], axis = 0) / (reshapeY*reshapeX) 
print(m_cov.shape)
values, vectors = eig(m_cov)
m_imgs1 = np.array([np.dot(vectors.T, i) for i in m_imgs])
print(m_imgs1.shape)


m_cov2 = np.sum([np.dot(i.T, i) for i in m_imgs1], axis = 0) / (reshapeY*reshapeX) 
print(m_cov2.shape)
values, vectors = eig(m_cov2)
m_imgs2 = np.array([np.dot(i, vectors) for i in m_imgs])
print(m_imgs2.shape)

112 92
(400, 112, 92)
(112, 112)
(400, 112, 92)
(92, 92)
(400, 112, 92)


In [111]:
def submatrix( matrix, startRow, startCol, size):
    return x[startRow:startRow+size,startCol:startCol+size]


In [116]:
# imgs_2dpca = np.array([i.reshape(reshapeY*reshapeX) for i in])
# imgs_2dpca = np.reshape(m_imgs2, (total_imgs, reshapeY*reshapeX))
imgs_2dpca = []
count = 0
for i in range(class_count):
    imgs_2dpca.append([])
    for j in range(image_count):
#         imgs_2dpca[i].append(np.reshape(m_imgs2[count], (reshapeY*reshapeX)))
        size_x = 6
        size_y = 6
        img = np.array([i[:size_x] for i in m_imgs2[count][:size_y]])
        imgs_2dpca[i].append(img)
#         imgs_2dpca[i].append(np.reshape(m_imgs2[count][:4][:4], (4*4)))
        count += 1
        
for i in range(1,test_count):
    count_to_load = i
    test, train = test_train(imgs_2dpca, count_to_load)
    result[i-1][3] = (fr(test, train))

0.5
0.55625
0.6107142857142857
0.6166666666666667
0.62
0.71875
0.7583333333333333
0.8375
0.85


In [117]:
mult = 1
reshapeY = len(rescale(imgs[0][0],mult))
reshapeX = len(rescale(imgs[0][0],mult)[0])
print(reshapeY, reshapeX)
# reshapeX = 31
imgs_prePCA = get_imgs_scaled(imgs, mult)
m_imgs = np.reshape(imgs_prePCA, (total_imgs , len(imgs_prePCA[0][0])))
m_imgs = np.array([np.reshape(i, (reshapeY, reshapeX)) for i in m_imgs])
print(m_imgs.shape)
# pca = get_pca(m_imgs)

m_cov = np.sum([np.dot(i, i.T) for i in m_imgs], axis = 0) / (reshapeY*reshapeX) 
m_cov2 = np.sum([np.dot(i.T, i) for i in m_imgs], axis = 0) / (reshapeY*reshapeX) 
print(m_cov.shape)
_, vectors = eig(m_cov)
_, vectors2 = eig(m_cov2)
m_imgs = np.array([np.dot(np.dot(vectors.T, i),  vectors2) for i in m_imgs])
print(m_imgs.shape)


# m_cov2 = np.sum([np.dot(i.T, i) for i in m_imgs], axis = 0) / (reshapeY*reshapeX) 
# print(m_cov2.shape)
# values, vectors = eig(m_cov2)
# m_imgs2 = np.array([np.dot(i, vectors) for i in m_imgs])
# print(m_imgs2.shape)

112 92
(400, 112, 92)
(112, 112)
(400, 112, 92)


In [118]:
imgs_2dpca = []
count = 0
for i in range(class_count):
    imgs_2dpca.append([])
    for j in range(image_count):
#         imgs_2dpca[i].append(np.reshape(m_imgs2[count], (reshapeY*reshapeX)))
        size_x = 6
        size_y = 6
        img = np.array([i[:size_x] for i in m_imgs[count][:size_y]])
        imgs_2dpca[i].append(img)
#         imgs_2dpca[i].append(np.reshape(m_imgs2[count][:4][:4], (4*4)))
        count += 1
        
for i in range(1,test_count):
    count_to_load = i
    test, train = test_train(imgs_2dpca, count_to_load)
    result[i-1][4] = (fr(test, train))

0.65
0.759375
0.7642857142857143
0.8
0.8150000000000001
0.9125
0.9416666666666667
0.9625
0.95


In [107]:
print(rescale(imgs[0][0],1).shape)

(112, 92)


In [78]:
a = np.array([[1, 0], [2, 3]])
print(np.dot(a, a.T))

[[ 1  2]
 [ 2 13]]


In [80]:
a = np.array([[1, 0], [2, 3]])
print(np.sum(a, axis = 0))

[3 3]


In [95]:
a = np.array([[1, 0, 5], [2, 3, 7], [7, 4, 2]])
print(submatrix())
print(a[:][:2])

[[1 0 5]
 [2 3 7]]
[[1 0 5]
 [2 3 7]]
